# Basic example

In [1]:
import seamless
seamless.delegate(False)

False

### 1. Import Seamless in IPython or Jupyter 

In [2]:
from seamless.workflow import Context
ctx = Context()

### 2. Set up a simple Seamless context

In [3]:
def add(a, b):
    return a + b

ctx.a = 10          # ctx.a => Seamless cell
ctx.b = 20          # ctx.b => Seamless cell
ctx.add = add       # ctx.add => Seamless transformer
ctx.add.a = ctx.a
ctx.add.b = ctx.b
ctx.c = ctx.add     # ctx.c => Seamless cell
await ctx.computation()
ctx.c.value

<Silk: 30 >

In [4]:
ctx.a += 5 
await ctx.computation()
ctx.c.value

<Silk: 35 >

### 3. Define schemas and validation rules

In [5]:
ctx.add.example.a = 0.0  # declares that add.a must be a number
ctx.add.example.b = 0.0  

def validate(self):
    assert self.a < self.b

ctx.add.add_validator(validate, name="validate")

await ctx.computation()
print(ctx.add.exception)
# Validation passes => exception is None

None


### 4. Create an API for a Seamless cell

In [6]:
def report(self): 
    value = self.unsilk 
    if value is None: 
        print("Sorry, there is no result") 
    else: 
        print("The result is: {}".format(value))

ctx.c.example.report = report
await ctx.computation()
ctx.c.value.report()

The result is: 35


### 5. Mount cells to the file system

In [7]:
ctx.a.celltype = "plain"
ctx.a.mount("a.json")
ctx.b.celltype = "plain"
ctx.b.mount("b.json")
ctx.c.celltype = "plain"
ctx.c.mount("c.json", mode="w")
ctx.add.code.mount("code.py")
await ctx.translation()

### 6. Share a cell over HTTP

In [8]:
ctx.c.mimetype = "text"
ctx.c.share()
await ctx.translation()

Opened the seamless share update server at port 5138
Opened the seamless REST server at port 5813


### http://localhost:5813/ctx/c

### 7. Control cells from Jupyter

In [9]:
from ipywidgets import IntSlider, IntText

a = IntSlider(min=-10,max=30)
b = IntSlider(min=-10,max=30)
c = ctx.c.output()
ctx.a.traitlet().link(a)
ctx.b.traitlet().link(b)
display(a)
display(b)
display(c)

IntSlider(value=15, max=30, min=-10)

IntSlider(value=20, max=30, min=-10)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '35'}, 'metadata': {}},))

### 8. Save the entire state of the context

In [10]:
# Graph and checksums, as JSON
ctx.save_graph("basic-example.seamless")
# Checksum-to-buffer cache, as ZIP file
ctx.save_zip("basic-example.zip")

### 9. In a new notebook / IPython console:
*(shutdown the current notebook first)*

```python
from seamless.workflow import load_graph
ctx = load_graph(
    "basic-example.seamless", 
    zip="basic-example.zip"
)
await ctx.computation()
ctx.c.value
```

35

### http://localhost:5813/ctx/c